## 1.Empty DataFrame

In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local").appName("Empty DataFrame").getOrCreate()
empty_df=spark.createDataFrame([],"id INT,name STRING")
empty_df.show()

+---+----+
| id|name|
+---+----+
+---+----+



## 2.RDD to DataFrame

In [0]:
rdd=spark.sparkContext.parallelize([(1,"Alice"),(2,"Bob")])
columns=["id","name"]
df_from_rdd=rdd.toDF(columns)
df_from_rdd.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 3.DF to Pandas

In [0]:
pandas_df=df_from_rdd.toPandas()
print(pandas_df)

   id   name
0   1  Alice
1   2    Bob


## 4.show()

In [0]:
df_from_rdd.show(5)

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 5.StructType & StructField

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

schema=StructType([StructField("id",IntegerType(),True),
                   StructField("name",StringType(),True)])

df_with_schema=spark.createDataFrame([(1,"Alice"),(2,"Bob")],schema)
df_with_schema.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 6.Column Class

In [0]:
from pyspark.sql import functions as F
df_with_column=df_from_rdd.withColumn("upper_name",F.upper(df_from_rdd['name']))
df_with_column.show()

+---+-----+----------+
| id| name|upper_name|
+---+-----+----------+
|  1|Alice|     ALICE|
|  2|  Bob|       BOB|
+---+-----+----------+



## 7.select()

In [0]:
df_from_rdd.select("id").show()

+---+
| id|
+---+
|  1|
|  2|
+---+



## 8.collect()

In [0]:
rows=df_from_rdd.collect()
print(rows)

[Row(id=1, name='Alice'), Row(id=2, name='Bob')]


## 9.withColumn()

In [0]:
df_with_new_col=df_from_rdd.withColumn("id_squared",df_from_rdd["id"]*df_from_rdd["id"])
df_with_new_col.show()

+---+-----+----------+
| id| name|id_squared|
+---+-----+----------+
|  1|Alice|         1|
|  2|  Bob|         4|
+---+-----+----------+



## 10.withColumnRenamed()

In [0]:
df_renamed=df_from_rdd.withColumnRenamed("name","full_name")
df_renamed.show()

+---+---------+
| id|full_name|
+---+---------+
|  1|    Alice|
|  2|      Bob|
+---+---------+



## 11.where() & filter()

In [0]:
df_filtered=df_from_rdd.where(df_from_rdd["id"]>1)
df_filtered.show()

df_filtered2=df_from_rdd.filter(df_from_rdd["id"]>1)
df_filtered2.show()

+---+----+
| id|name|
+---+----+
|  2| Bob|
+---+----+

+---+----+
| id|name|
+---+----+
|  2| Bob|
+---+----+



## 12.drop() & dropDuplicates()

In [0]:
df_dropped=df_from_rdd.drop("name")
df_dropped.show()

df_no_duplicates=df_from_rdd.dropDuplicates()
df_no_duplicates.show()

+---+
| id|
+---+
|  1|
|  2|
+---+

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 13.orderBY() and Sort()

In [0]:
df_sorted=df_from_rdd.orderBy("id",ascending=False)
df_sorted.show()

df_sorted2=df_from_rdd.sort("id")
df_sorted2.show()

+---+-----+
| id| name|
+---+-----+
|  2|  Bob|
|  1|Alice|
+---+-----+

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 14.groupBy()

In [0]:
df_groupped=df_from_rdd.groupBy("id").count()
df_groupped.show()

+---+-----+
| id|count|
+---+-----+
|  1|    1|
|  2|    1|
+---+-----+



## 15.join()

In [0]:
df2=spark.createDataFrame([(1,'Math'),(2,'Science')],['id','subject'])
df_joined=df_from_rdd.join(df2,on="id")
df_joined.show()

+---+-----+-------+
| id| name|subject|
+---+-----+-------+
|  1|Alice|   Math|
|  2|  Bob|Science|
+---+-----+-------+



## 16.union() & unionAll()

In [0]:
df3=spark.createDataFrame([(3,"charlie")],["id","name"])
df_union=df_from_rdd.union(df3)
df_union.show()


df4=spark.createDataFrame([(3,"charlie"),(3,"charlie")],["id","name"])
df_union_all=df_from_rdd.unionAll(df4)
df_union_all.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|charlie|
+---+-------+

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|charlie|
|  3|charlie|
+---+-------+



## 17.unionByName()

In [0]:
df_union_by_name=df_from_rdd.unionByName(df3) #column names must be same
df_union_by_name.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|charlie|
+---+-------+



## 18.UDF()

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def add_eclamation(name):
  return name + "!"

add_udf=udf(add_eclamation,StringType())
df_udf=df_from_rdd.withColumn("excited_name",add_udf("name"))
df_udf.show()

+---+-----+------------+
| id| name|excited_name|
+---+-----+------------+
|  1|Alice|      Alice!|
|  2|  Bob|        Bob!|
+---+-----+------------+



## 19.transform()

In [0]:
df_transformed=df_from_rdd.transform(lambda df:df.withColumn("id_squared",df["id"]**2))
df_transformed.show()

+---+-----+----------+
| id| name|id_squared|
+---+-----+----------+
|  1|Alice|       1.0|
|  2|  Bob|       4.0|
+---+-----+----------+



## 20.apply()

In [0]:
df_applied = df_from_rdd.rdd.map(lambda row: (row.id * 2, row.name)).toDF(["id", "name"])
df_applied.show()

+---+-----+
| id| name|
+---+-----+
|  2|Alice|
|  4|  Bob|
+---+-----+



## 21.map()

In [0]:
rdd_mapped=df_from_rdd.rdd.map(lambda x:(x.id*2,x.name))
df_mapped=rdd_mapped.toDF(["id","name"])
df_mapped.show()

+---+-----+
| id| name|
+---+-----+
|  2|Alice|
|  4|  Bob|
+---+-----+



## 22.flatMap()

In [0]:
rdd_flat=df_from_rdd.rdd.flatMap(lambda x:[(x.id,x.name),(x.id*10,x.name)])
df_flat=rdd_flat.toDF(["id","name"])
df_flat.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
| 10|Alice|
|  2|  Bob|
| 20|  Bob|
+---+-----+



## 23.foreach()

In [0]:
def print_row(row):
  print(row)
  
df_from_rdd.foreach(print_row)

## 24.sample() vs sampleBy()

In [0]:
df_sample=df_from_rdd.sample(fraction=0.5)
df_sample.show()

df_sample_by=df_from_rdd.sampleBy("id",fractions={1:0.5,2:0.5})
df_sample_by.show()

+---+----+
| id|name|
+---+----+
|  2| Bob|
+---+----+

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
+---+-----+



## 25.fillna() & fill()

In [0]:
df_filled=df_from_rdd.fillna({'id':0,'name':'Unknown'})
df_filled.show()

# df_filled2=df_from_rdd.fill({'id':0,'name':'Unknown'})
# df_filled2.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 26.pivot()(Row to Column)

In [0]:
df_pivoted=df_from_rdd.groupBy("id").pivot("name").agg({"id":"count"})
df_pivoted.show()

+---+-----+----+
| id|Alice| Bob|
+---+-----+----+
|  1|    1|NULL|
|  2| NULL|   1|
+---+-----+----+



## 27.partitionBy()

In [0]:
df_partitioned=df_from_rdd.repartitionByRange(3,"id")
df_partitioned.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 28. MapType (Map/Dict)

In [0]:
from pyspark.sql.types import MapType,StringType,StructType,StructField,IntegerType
data = [(1, {"name": "Alice", "age": "25"}), (2, {"name": "Bob", "age": "30"})]
schema=StructType([StructField("id",IntegerType(),True),StructField("info",MapType(StringType(),StringType()),True)])
df_map=spark.createDataFrame(data,schema)
df_map.show()

+---+--------------------+
| id|                info|
+---+--------------------+
|  1|{name -> Alice, a...|
|  2|{name -> Bob, age...|
+---+--------------------+



#Inbuilt-Functions

##29.col()

In [0]:
df=df_from_rdd
df.select(F.col('id')).show()


+---+
| id|
+---+
|  1|
|  2|
+---+



##30.lit()

In [0]:
df.withColumn('constant',F.lit(10)).show()

+---+-----+--------+
| id| name|constant|
+---+-----+--------+
|  1|Alice|      10|
|  2|  Bob|      10|
+---+-----+--------+



##31.when()

In [0]:
df.withColumn('is_adult',F.when(F.col('id')>1,'Yes').otherwise('No')).show()

+---+-----+--------+
| id| name|is_adult|
+---+-----+--------+
|  1|Alice|      No|
|  2|  Bob|     Yes|
+---+-----+--------+



##32. isNull():


In [0]:
df.filter(F.col('name').isNull()).show()

+---+----+
| id|name|
+---+----+
+---+----+



##33.isNotNull()

In [0]:
df.filter(F.col('name').isNotNull()).show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



##34.between()

In [0]:
df.filter(F.col('id').between(2,3)).show()

+---+----+
| id|name|
+---+----+
|  2| Bob|
+---+----+



##35.like()

In [0]:
df.filter(F.col('name').like('A%')).show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
+---+-----+



##36.rlike()

In [0]:
df.filter(F.col('name').rlike('A.*')).show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
+---+-----+



##37.alias()

In [0]:
df.select(F.col('name').alias('user_name')).show()


+---------+
|user_name|
+---------+
|    Alice|
|      Bob|
+---------+



##38.cast()

In [0]:
df.withColumn('id_str', F.col('id').cast('string')).show()

+---+-----+------+
| id| name|id_str|
+---+-----+------+
|  1|Alice|     1|
|  2|  Bob|     2|
+---+-----+------+



##39.expr()

In [0]:
df.select(F.expr('id + 1')).show()


+--------+
|(id + 1)|
+--------+
|       2|
|       3|
+--------+



##40.count()

In [0]:
df.select(F.count('id')).show()

+---------+
|count(id)|
+---------+
|        2|
+---------+



##41.countDistinct()

In [0]:
df.select(F.countDistinct('name')).show()

+--------------------+
|count(DISTINCT name)|
+--------------------+
|                   2|
+--------------------+



##42.approx_count_distinct()

In [0]:
df.select(F.approx_count_distinct('name')).show()

+---------------------------+
|approx_count_distinct(name)|
+---------------------------+
|                          2|
+---------------------------+



##43.sum()

In [0]:
df.select(F.sum('id')).show()

+-------+
|sum(id)|
+-------+
|      3|
+-------+



##44.sumDistinct()

In [0]:
df.select(F.sum_distinct('id')).show()

+----------------+
|sum(DISTINCT id)|
+----------------+
|               3|
+----------------+



##45.avg()

In [0]:
df.select(F.avg('id')).show()

+-------+
|avg(id)|
+-------+
|    1.5|
+-------+



##46.min()

In [0]:
df.select(F.min('id')).show()

+-------+
|min(id)|
+-------+
|      1|
+-------+



##47.max()

In [0]:
df.select(F.max('id')).show()

+-------+
|max(id)|
+-------+
|      2|
+-------+



##48.first()

In [0]:
df.select(F.first('id')).show()

+---------+
|first(id)|
+---------+
|        1|
+---------+



##49.last()

In [0]:
df.select(F.last('id')).show()

+--------+
|last(id)|
+--------+
|       2|
+--------+



##50.stddev()

In [0]:
df.select(F.stddev('id')).show()

+------------------+
|        stddev(id)|
+------------------+
|0.7071067811865476|
+------------------+



##51.variance()

In [0]:
df.select(F.variance('id')).show()

+------------+
|variance(id)|
+------------+
|         0.5|
+------------+



##52.collect_list()

In [0]:
df.groupBy('name').agg(F.collect_list('id')).show()

+-----+----------------+
| name|collect_list(id)|
+-----+----------------+
|Alice|             [1]|
|  Bob|             [2]|
+-----+----------------+



##53.collect_set()

In [0]:
df.groupBy('name').agg(F.collect_set('id')).show()

+-----+---------------+
| name|collect_set(id)|
+-----+---------------+
|Alice|            [1]|
|  Bob|            [2]|
+-----+---------------+



##54.corr()

In [0]:
dff=df.withColumn('id_squared', F.col('id')*F.col('id'))
dff.select(F.corr('id', 'id_squared')).show()

+--------------------+
|corr(id, id_squared)|
+--------------------+
|                 1.0|
+--------------------+



##55.covar_pop()

In [0]:
dff.select(F.covar_pop('id','id_squared')).show()

+-------------------------+
|covar_pop(id, id_squared)|
+-------------------------+
|                     0.75|
+-------------------------+



##56.covar_samp()

In [0]:
dff.select(F.covar_samp('id','id_squared')).show()

+--------------------------+
|covar_samp(id, id_squared)|
+--------------------------+
|                       1.5|
+--------------------------+



##57.kurtosis()

In [0]:
df.select(F.kurtosis('id')).show()

+------------+
|kurtosis(id)|
+------------+
|        -2.0|
+------------+



##58.skewness()

In [0]:
df.select(F.skewness('id')).show()

+------------+
|skewness(id)|
+------------+
|         0.0|
+------------+



##59.approxQuantile()

In [0]:
df.approxQuantile('id', [0.25, 0.5, 0.75], 0.1)

[1.0, 1.0, 2.0]

##60.rank()

In [0]:
from pyspark.sql.window import Window
windowSpec = Window.orderBy('id')
df_union_all.withColumn('rank', F.rank().over(windowSpec)).show()

+---+-------+----+
| id|   name|rank|
+---+-------+----+
|  1|  Alice|   1|
|  2|    Bob|   2|
|  3|charlie|   3|
|  3|charlie|   3|
+---+-------+----+



##61.dense_rank()

In [0]:
df_union_all.withColumn('dense_rank', F.dense_rank().over(windowSpec)).show()

+---+-------+----------+
| id|   name|dense_rank|
+---+-------+----------+
|  1|  Alice|         1|
|  2|    Bob|         2|
|  3|charlie|         3|
|  3|charlie|         3|
+---+-------+----------+



##62.ntile()

In [0]:
df_union_all.withColumn('ntile', F.ntile(2).over(windowSpec)).show()

+---+-------+-----+
| id|   name|ntile|
+---+-------+-----+
|  1|  Alice|    1|
|  2|    Bob|    1|
|  3|charlie|    2|
|  3|charlie|    2|
+---+-------+-----+



##63.row_number()

In [0]:
df_union_all.withColumn('row_number', F.row_number().over(windowSpec)).show()

+---+-------+----------+
| id|   name|row_number|
+---+-------+----------+
|  1|  Alice|         1|
|  2|    Bob|         2|
|  3|charlie|         3|
|  3|charlie|         4|
+---+-------+----------+



##64.lead()

In [0]:
df_union_all.withColumn('next_id', F.lead('id').over(windowSpec)).show()

+---+-------+-------+
| id|   name|next_id|
+---+-------+-------+
|  1|  Alice|      2|
|  2|    Bob|      3|
|  3|charlie|      3|
|  3|charlie|   NULL|
+---+-------+-------+



##65.lag()

In [0]:
df_union_all.withColumn('prev_id', F.lag('id').over(windowSpec)).show()

+---+-------+-------+
| id|   name|prev_id|
+---+-------+-------+
|  1|  Alice|   NULL|
|  2|    Bob|      1|
|  3|charlie|      2|
|  3|charlie|      3|
+---+-------+-------+



##66.percent_rank()

In [0]:
df_union_all.withColumn('percent_rank',F.percent_rank().over(windowSpec)).show()

+---+-------+------------------+
| id|   name|      percent_rank|
+---+-------+------------------+
|  1|  Alice|               0.0|
|  2|    Bob|0.3333333333333333|
|  3|charlie|0.6666666666666666|
|  3|charlie|0.6666666666666666|
+---+-------+------------------+



##67.window()

This function is used to define a window specification, which is typically used in conjunction with window
functions such as rank(), dense_rank(), row_number(), etc. This has already been shown in some examples
above (e.g., Window.orderBy('id')).

# String Functions

## 68.concat()

In [0]:
df.withColumn('Hello', F.concat(F.lit('Hello '), F.col('name'))).show()

+---+-----+-----------+
| id| name|      Hello|
+---+-----+-----------+
|  1|Alice|Hello Alice|
|  2|  Bob|  Hello Bob|
+---+-----+-----------+



## 69.concat_ws()

In [0]:
df.withColumn('full_name', F.concat_ws(' ', F.col('name'), F.lit('Doe'))).show()

+---+-----+---------+
| id| name|full_name|
+---+-----+---------+
|  1|Alice|Alice Doe|
|  2|  Bob|  Bob Doe|
+---+-----+---------+



## 70.length()

In [0]:
df.withColumn('name_length', F.length(F.col('name'))).show()

+---+-----+-----------+
| id| name|name_length|
+---+-----+-----------+
|  1|Alice|          5|
|  2|  Bob|          3|
+---+-----+-----------+



## 71.lower()

In [0]:
df.withColumn('name_lower', F.lower(F.col('name'))).show()

+---+-----+----------+
| id| name|name_lower|
+---+-----+----------+
|  1|Alice|     alice|
|  2|  Bob|       bob|
+---+-----+----------+



## 72.upper()

In [0]:
df.withColumn('name_lower', F.upper(F.col('name'))).show()

+---+-----+----------+
| id| name|name_lower|
+---+-----+----------+
|  1|Alice|     ALICE|
|  2|  Bob|       BOB|
+---+-----+----------+



## 73.trim()

In [0]:
df.withColumn('trim_name', F.trim(F.col('name'))).show()

+---+-----+---------+
| id| name|trim_name|
+---+-----+---------+
|  1|Alice|    Alice|
|  2|  Bob|      Bob|
+---+-----+---------+



## 74.ltrim()

In [0]:
df.withColumn('ltrim_name', F.ltrim(F.col('name'))).show()

+---+-----+----------+
| id| name|ltrim_name|
+---+-----+----------+
|  1|Alice|     Alice|
|  2|  Bob|       Bob|
+---+-----+----------+



## 75.rtrim()

In [0]:
df.withColumn('rtrim_name', F.rtrim(F.col('name'))).show()

+---+-----+----------+
| id| name|rtrim_name|
+---+-----+----------+
|  1|Alice|     Alice|
|  2|  Bob|       Bob|
+---+-----+----------+



## 76.reverse()

In [0]:
df.withColumn('reversed_name', F.reverse(F.col('name'))).show()

+---+-----+-------------+
| id| name|reversed_name|
+---+-----+-------------+
|  1|Alice|        ecilA|
|  2|  Bob|          boB|
+---+-----+-------------+



## 77.substring()

In [0]:
df.withColumn('sub_name', F.substring(F.col('name'), 1, 3)).show()

+---+-----+--------+
| id| name|sub_name|
+---+-----+--------+
|  1|Alice|     Ali|
|  2|  Bob|     Bob|
+---+-----+--------+



## 78.substr():

In [0]:
df.withColumn('substr_name', F.substring(F.col('name'), 1, 3)).show()


+---+-----+-----------+
| id| name|substr_name|
+---+-----+-----------+
|  1|Alice|        Ali|
|  2|  Bob|        Bob|
+---+-----+-----------+



## 79.split()

In [0]:
df.withColumn('split_name', F.split(F.col('name'), 'l')).show()

+---+-----+----------+
| id| name|split_name|
+---+-----+----------+
|  1|Alice|  [A, ice]|
|  2|  Bob|     [Bob]|
+---+-----+----------+



## 80.regexp_extract()

In [0]:
df.withColumn('name_initial', F.regexp_extract(F.col('name'), '^(.)', 0)).show()

+---+-----+------------+
| id| name|name_initial|
+---+-----+------------+
|  1|Alice|           A|
|  2|  Bob|           B|
+---+-----+------------+



## 81.regexp_replace():

In [0]:
df.withColumn('name_replaced', F.regexp_replace(F.col('name'), 'i', 'X')).show()

+---+-----+-------------+
| id| name|name_replaced|
+---+-----+-------------+
|  1|Alice|        AlXce|
|  2|  Bob|          Bob|
+---+-----+-------------+



## 82.instr()

In [0]:
df.withColumn('name_pos', F.instr(F.col('name'), 'i')).show()

+---+-----+--------+
| id| name|name_pos|
+---+-----+--------+
|  1|Alice|       3|
|  2|  Bob|       0|
+---+-----+--------+



## 83.translate()

In [0]:
df.withColumn('translated_name', F.translate(F.col('name'), 'Ae', 'XY')).show()

+---+-----+---------------+
| id| name|translated_name|
+---+-----+---------------+
|  1|Alice|          XlicY|
|  2|  Bob|            Bob|
+---+-----+---------------+



## 84.encode()

In [0]:
df.withColumn('encoded_name', F.encode(F.col('name'), 'UTF-8')).show()

+---+-----+----------------+
| id| name|    encoded_name|
+---+-----+----------------+
|  1|Alice|[41 6C 69 63 65]|
|  2|  Bob|      [42 6F 62]|
+---+-----+----------------+



## 85.decode()

In [0]:
en=df.withColumn('encoded_name', F.encode(F.col('name'), 'UTF-8'))
en.withColumn('decoded_name', F.decode(F.col('encoded_name'), 'UTF-8')).show()

+---+-----+----------------+------------+
| id| name|    encoded_name|decoded_name|
+---+-----+----------------+------------+
|  1|Alice|[41 6C 69 63 65]|       Alice|
|  2|  Bob|      [42 6F 62]|         Bob|
+---+-----+----------------+------------+



## 86.overlay()

In [0]:
df.withColumn('overlay_name', F.overlay(F.col('name'), F.lit('x'), 2, 3)).show()

+---+-----+------------+
| id| name|overlay_name|
+---+-----+------------+
|  1|Alice|         Axe|
|  2|  Bob|          Bx|
+---+-----+------------+



## 87.format_number()

In [0]:
df.withColumn('formatted_id', F.format_number(F.col('id'), 2)).show()

+---+-----+------------+
| id| name|formatted_id|
+---+-----+------------+
|  1|Alice|        1.00|
|  2|  Bob|        2.00|
+---+-----+------------+



## 88.initcap()

In [0]:
df.withColumn('initcap_name', F.initcap(F.col('name'))).show()

+---+-----+------------+
| id| name|initcap_name|
+---+-----+------------+
|  1|Alice|       Alice|
|  2|  Bob|         Bob|
+---+-----+------------+



## 89.repeat()

In [0]:
df.withColumn('repeated_name', F.repeat(F.col('name'), 3)).show()

+---+-----+---------------+
| id| name|  repeated_name|
+---+-----+---------------+
|  1|Alice|AliceAliceAlice|
|  2|  Bob|      BobBobBob|
+---+-----+---------------+



## 90.rpad()

In [0]:
df.withColumn('padded_name', F.rpad(F.col('name'), 10, 'X')).show()

+---+-----+-----------+
| id| name|padded_name|
+---+-----+-----------+
|  1|Alice| AliceXXXXX|
|  2|  Bob| BobXXXXXXX|
+---+-----+-----------+



## 91.lpad()

In [0]:
df.withColumn('padded_name', F.lpad(F.col('name'), 10, 'X')).show()

+---+-----+-----------+
| id| name|padded_name|
+---+-----+-----------+
|  1|Alice| XXXXXAlice|
|  2|  Bob| XXXXXXXBob|
+---+-----+-----------+



## 92.trim()

In [0]:
df.withColumn('trim_name', F.trim(F.col('name'))).show()

+---+-----+---------+
| id| name|trim_name|
+---+-----+---------+
|  1|Alice|    Alice|
|  2|  Bob|      Bob|
+---+-----+---------+



## 93.soundex()

In [0]:
df.withColumn('soundex_name', F.soundex(F.col('name'))).show()

+---+-----+------------+
| id| name|soundex_name|
+---+-----+------------+
|  1|Alice|        A420|
|  2|  Bob|        B100|
+---+-----+------------+



## 94.contains()

In [0]:
df.withColumn('contains', F.expr("contains(name, 'Alic')")).show()

+---+-----+--------+
| id| name|contains|
+---+-----+--------+
|  1|Alice|    true|
|  2|  Bob|   false|
+---+-----+--------+



# DateTimeFunctions

## 95.current_date():

In [0]:
df.withColumn('current_date', F.current_date()).show()

+---+-----+------------+
| id| name|current_date|
+---+-----+------------+
|  1|Alice|  2025-02-13|
|  2|  Bob|  2025-02-13|
+---+-----+------------+



## 96.current_timestamp()

In [0]:
df.withColumn('current_timestamp',F.current_timestamp()).show()

+---+-----+--------------------+
| id| name|   current_timestamp|
+---+-----+--------------------+
|  1|Alice|2025-02-13 19:06:...|
|  2|  Bob|2025-02-13 19:06:...|
+---+-----+--------------------+



## 97.date_add()

In [0]:
df.withColumn('date_plus_5', F.date_add(F.current_date(), 5)).show()

+---+-----+-----------+
| id| name|date_plus_5|
+---+-----+-----------+
|  1|Alice| 2025-02-18|
|  2|  Bob| 2025-02-18|
+---+-----+-----------+



## 98.date_sub()

In [0]:
df.withColumn('date_minus_5',F.date_sub(F.current_date(),5)).show()

+---+-----+------------+
| id| name|date_minus_5|
+---+-----+------------+
|  1|Alice|  2025-02-08|
|  2|  Bob|  2025-02-08|
+---+-----+------------+



## 99.datediff()

In [0]:
df.withColumn('days_',F.datediff(F.lit('2025-01-01'), F.current_date())).show()

+---+-----+-----+
| id| name|days_|
+---+-----+-----+
|  1|Alice|  -43|
|  2|  Bob|  -43|
+---+-----+-----+



## 100.to_date()

In [0]:
df.withColumn('date_from_string', F.to_date(F.lit('2025-01-01'))).show()

+---+-----+----------------+
| id| name|date_from_string|
+---+-----+----------------+
|  1|Alice|      2025-01-01|
|  2|  Bob|      2025-01-01|
+---+-----+----------------+



## 101.to_timestamp()

In [0]:
df.withColumn('timestamp_from_string', F.to_timestamp(F.lit('2025-01-01 10:00:00'))).show()

+---+-----+---------------------+
| id| name|timestamp_from_string|
+---+-----+---------------------+
|  1|Alice|  2025-01-01 10:00:00|
|  2|  Bob|  2025-01-01 10:00:00|
+---+-----+---------------------+



## 102.rom_unixtime()

In [0]:
ut=df.withColumn('unix_timestamp_value', F.unix_timestamp(F.lit('2025-01-01 10:00:00')))
ut.withColumn('timestamp_from_unix', F.from_unixtime(F.lit(1674790520))).show()

+---+-----+--------------------+-------------------+
| id| name|unix_timestamp_value|timestamp_from_unix|
+---+-----+--------------------+-------------------+
|  1|Alice|          1735725600|2023-01-27 03:35:20|
|  2|  Bob|          1735725600|2023-01-27 03:35:20|
+---+-----+--------------------+-------------------+



## 103.unix_timestamp()

In [0]:
df.withColumn('unix_timestamp_value', F.unix_timestamp(F.lit('2025-01-01 10:00:00'))).show()

+---+-----+--------------------+
| id| name|unix_timestamp_value|
+---+-----+--------------------+
|  1|Alice|          1735725600|
|  2|  Bob|          1735725600|
+---+-----+--------------------+



## 104.year()

In [0]:
df.withColumn('year_extracted', F.year(F.col('current_date'))).show()

+---+-----+--------------+
| id| name|year_extracted|
+---+-----+--------------+
|  1|Alice|          2025|
|  2|  Bob|          2025|
+---+-----+--------------+



## 105.month()

In [0]:
df.withColumn('month_extracted', F.month(F.col('current_date'))).show()

+---+-----+---------------+
| id| name|month_extracted|
+---+-----+---------------+
|  1|Alice|              2|
|  2|  Bob|              2|
+---+-----+---------------+



## 106.dayofmonth():

In [0]:
df.withColumn('day_of_month', F.dayofmonth(F.col('current_date'))).show()

+---+-----+------------+
| id| name|day_of_month|
+---+-----+------------+
|  1|Alice|          13|
|  2|  Bob|          13|
+---+-----+------------+



## 107.dayofweek()

In [0]:
df.withColumn('day_of_week', F.dayofweek(F.col('current_date'))).show()

+---+-----+-----------+
| id| name|day_of_week|
+---+-----+-----------+
|  1|Alice|          5|
|  2|  Bob|          5|
+---+-----+-----------+



## 108.dayofyear()

In [0]:
df.withColumn('day_of_year', F.dayofyear(F.col('current_date'))).show()

+---+-----+-----------+
| id| name|day_of_year|
+---+-----+-----------+
|  1|Alice|         44|
|  2|  Bob|         44|
+---+-----+-----------+



## 109.hour()

In [0]:
df.withColumn('hour_extracted', F.hour(F.col('current_timestamp'))).show()

+---+-----+--------------+
| id| name|hour_extracted|
+---+-----+--------------+
|  1|Alice|            19|
|  2|  Bob|            19|
+---+-----+--------------+



## 110.minute()

In [0]:
df.withColumn('minute_extracted', F.minute(F.col('current_timestamp'))).show()

+---+-----+----------------+
| id| name|minute_extracted|
+---+-----+----------------+
|  1|Alice|               6|
|  2|  Bob|               6|
+---+-----+----------------+



## 111.second()

In [0]:
df.withColumn('second_extracted', F.second(F.col('current_timestamp'))).show()

+---+-----+----------------+
| id| name|second_extracted|
+---+-----+----------------+
|  1|Alice|              48|
|  2|  Bob|              48|
+---+-----+----------------+



## 112.date_format()

In [0]:
df.withColumn('formatted_date', F.date_format(F.col('current_date'), 'yyyy-MM-dd')).show()

+---+-----+--------------+
| id| name|formatted_date|
+---+-----+--------------+
|  1|Alice|    2025-02-13|
|  2|  Bob|    2025-02-13|
+---+-----+--------------+



## 113.last_day()

In [0]:
df.withColumn('last_day_of_month', F.last_day(F.col('current_date'))).show()

+---+-----+-----------------+
| id| name|last_day_of_month|
+---+-----+-----------------+
|  1|Alice|       2025-02-28|
|  2|  Bob|       2025-02-28|
+---+-----+-----------------+



## 114.next_day()

In [0]:
df.withColumn('next_monday', F.next_day(F.col('current_date'), 'Monday')).show()

+---+-----+-----------+
| id| name|next_monday|
+---+-----+-----------+
|  1|Alice| 2025-02-17|
|  2|  Bob| 2025-02-17|
+---+-----+-----------+



## 115.trunc()

In [0]:
df.withColumn('truncated_date', F.trunc(F.col('current_date'), 'MM')).show()

+---+-----+--------------+
| id| name|truncated_date|
+---+-----+--------------+
|  1|Alice|    2025-02-01|
|  2|  Bob|    2025-02-01|
+---+-----+--------------+



## 116.add_months()

In [0]:
df.withColumn('date_plus_2_months', F.add_months(F.col('current_date'), 2)).show()

+---+-----+------------------+
| id| name|date_plus_2_months|
+---+-----+------------------+
|  1|Alice|        2025-04-13|
|  2|  Bob|        2025-04-13|
+---+-----+------------------+



## 117.months_between():

In [0]:
df.withColumn('months_', F.months_between(F.col('current_date'), F.lit('2025-01-01'))).show()

+---+-----+----------+
| id| name|   months_|
+---+-----+----------+
|  1|Alice|1.38709677|
|  2|  Bob|1.38709677|
+---+-----+----------+



## 118.weekofyear():

In [0]:
df.withColumn('week_of_year', F.weekofyear(F.col('current_date'))).show()

+---+-----+------------+
| id| name|week_of_year|
+---+-----+------------+
|  1|Alice|           7|
|  2|  Bob|           7|
+---+-----+------------+



## 119.timestamp()

In [0]:
df.withColumn('timestamp_example', F.to_timestamp(F.lit('2025-01-01 10:00:00'))).show()

+---+-----+-------------------+
| id| name|  timestamp_example|
+---+-----+-------------------+
|  1|Alice|2025-01-01 10:00:00|
|  2|  Bob|2025-01-01 10:00:00|
+---+-----+-------------------+



## 120.date_trunc()

In [0]:
df.withColumn('date_trunc_year', F.date_trunc('YEAR', F.col('current_date'))).show()

+---+-----+-------------------+
| id| name|    date_trunc_year|
+---+-----+-------------------+
|  1|Alice|2025-01-01 00:00:00|
|  2|  Bob|2025-01-01 00:00:00|
+---+-----+-------------------+



# Array

## 121.array()

In [0]:
df.withColumn('array_example', F.array('id', 'name')).show()

+---+-----+-------------+
| id| name|array_example|
+---+-----+-------------+
|  1|Alice|   [1, Alice]|
|  2|  Bob|     [2, Bob]|
+---+-----+-------------+



## 122.array_contains()

In [0]:
arr=df.withColumn('array_example', F.array('id', 'name'))
arr.withColumn('contains_alice', F.array_contains(F.col('array_example'), 'Alice')).show()

+---+-----+-------------+--------------+
| id| name|array_example|contains_alice|
+---+-----+-------------+--------------+
|  1|Alice|   [1, Alice]|          true|
|  2|  Bob|     [2, Bob]|         false|
+---+-----+-------------+--------------+



## 123.array_distinct()

In [0]:
df.withColumn('distinct_array', F.array_distinct(F.array(F.lit(1), F.lit(2), F.lit(1), F.lit(3)))).show()

+---+-----+--------------+
| id| name|distinct_array|
+---+-----+--------------+
|  1|Alice|     [1, 2, 3]|
|  2|  Bob|     [1, 2, 3]|
+---+-----+--------------+



## 124.array_intersect()

In [0]:
df.withColumn('array_intersect', F.array_intersect(F.array(F.lit(1), F.lit(2), F.lit(3)), F.array(F.lit(1), F.lit(3),
F.lit(4)))).show()

+---+-----+---------------+
| id| name|array_intersect|
+---+-----+---------------+
|  1|Alice|         [1, 3]|
|  2|  Bob|         [1, 3]|
+---+-----+---------------+



## 125.array_union()

In [0]:
df.withColumn('array_union', F.array_union(F.array(F.lit(1), F.lit(2), F.lit(3)), F.array(F.lit(3), F.lit(4),
F.lit(5)))).show()

+---+-----+---------------+
| id| name|    array_union|
+---+-----+---------------+
|  1|Alice|[1, 2, 3, 4, 5]|
|  2|  Bob|[1, 2, 3, 4, 5]|
+---+-----+---------------+

